In [1]:
import pandas as pd
import numpy as np

In [2]:
monday_df    = pd.read_csv('./data/monday.csv', sep=';', parse_dates=True, index_col=0)
tuesday_df   = pd.read_csv('./data/tuesday.csv', sep=';', parse_dates=True, index_col=0)
wednesday_df = pd.read_csv('./data/wednesday.csv', sep=';', parse_dates=True, index_col=0)
thusday_df   = pd.read_csv('./data/thursday.csv', sep=';', parse_dates=True, index_col=0)
friday_df    = pd.read_csv('./data/friday.csv', sep=';', parse_dates=True, index_col=0)
monday    = monday_df.copy()
tuesday   = tuesday_df.copy()
wednesday = wednesday_df.copy()
thursday  = thusday_df.copy()
friday    = friday_df.copy()
monday   ['day'] ='monday'
tuesday  ['day'] ='tuesday'
wednesday['day'] ='wednesday'
thursday ['day'] ='thursday'
friday   ['day'] ='friday'




days=[monday, tuesday, wednesday, thursday, friday]

In [3]:
customer_num_per_day=[]
for day in days:
    customer_num_per_day.append(day['customer_no'].max())

monday_cp    = monday_df.copy()
tuesday_cp   = tuesday_df.copy()
wednesday_cp = wednesday_df.copy()
thursday_cp  = thusday_df.copy()
friday_cp    = friday_df.copy()
cp_days      = [monday_cp, tuesday_cp, wednesday_cp, thursday_cp, friday_cp]

sum(customer_num_per_day)/5

1489.0

In [4]:
for i, cp_day in enumerate(cp_days):
    cp_day['customer_no'] = cp_day['customer_no'] + sum(customer_num_per_day[:i])
    cp_days[i] = cp_day
days.append(pd.concat(cp_days))

In [5]:
days

[                     customer_no  location     day
 timestamp                                         
 2019-09-02 07:03:00            1     dairy  monday
 2019-09-02 07:03:00            2     dairy  monday
 2019-09-02 07:04:00            3     dairy  monday
 2019-09-02 07:04:00            4     dairy  monday
 2019-09-02 07:04:00            5    spices  monday
 ...                          ...       ...     ...
 2019-09-02 21:49:00         1442  checkout  monday
 2019-09-02 21:49:00         1444  checkout  monday
 2019-09-02 21:49:00         1445     dairy  monday
 2019-09-02 21:50:00         1446     dairy  monday
 2019-09-02 21:50:00         1447     fruit  monday
 
 [4884 rows x 3 columns],
                      customer_no  location      day
 timestamp                                          
 2019-09-03 07:02:00            1     fruit  tuesday
 2019-09-03 07:02:00            2    drinks  tuesday
 2019-09-03 07:03:00            3     dairy  tuesday
 2019-09-03 07:05:00           

In [6]:
def time_step(df):
    df=df.groupby('customer_no').resample('1Min').ffill()
    df = df.drop(columns=['customer_no'])
    #df.reset_index(inplace=True)
    return df

In [7]:
def to_from(df):
    df['from'] = df['location']
    df['to']   = df['location'].shift(1)
    df.drop(columns='location', inplace=True)
    df.fillna(value='checkout', inplace=True)
    return df

In [8]:
def kickout(df):
    for idk in range(1, df['customer_no'].max()+1):
        if df.loc[df['customer_no']==idk, 'from'].iloc[-1] != 'checkout':
            print('Customer#:  ', idk,'   kickout!')
            temp_df = df.loc[df['customer_no']==idk].iloc[-1].to_frame().T
            temp_df.loc[temp_df['customer_no'] == idk,'from']     = temp_df.loc[temp_df['customer_no'] == idk,'to']
            temp_df.loc[temp_df['customer_no'] == idk,'to'] = 'checkout'
            temp_ts = temp_df.loc[temp_df['customer_no'] == idk]['timestamp']
            temp_ts = temp_ts + pd.DateOffset(minutes=1)
            #print(temp_df.loc[temp_df['customer_no'] == idk]['timestamp'] + pd.DateOffset(minutes=1))
            df=pd.concat([df, temp_df], ignore_index=True)

In [9]:
tm_list=[]
def get_tm(df):
    df=df.sort_values(by=['timestamp']).groupby('customer_no')
    tm = pd.crosstab(monday['to'], monday['from'], normalize='index')
    tm.loc['checkout'] = [1, 0, 0, 0, 0]
    return tm

In [10]:
for i, day in enumerate(days):
    #print('DATAFRAME ', day['day'],' is in process! ......')
    print('1------> getting timesteps')#,day.head(1))
    days[i]    = time_step(day)
    print('2------> destinations')#,day.head(1))
    days[i]    = to_from(day).reset_index(inplace=True)
    day['timestamp']=pd.to_datetime(day['timestamp'])
    print('3------> kickout')#,day.head(1))
    days[i]    = kickout(day)
    print('4------> Creating TM')
    tm_list.append(get_tm(day))
    #print(tm_list[i])
    print(day)
    print('--------------------------------------------------------------------------------------------->')

1------> getting timesteps
2------> destinations
3------> kickout
Customer#:   1430    kickout!
Customer#:   1433    kickout!
Customer#:   1437    kickout!
Customer#:   1439    kickout!
Customer#:   1440    kickout!
Customer#:   1441    kickout!
Customer#:   1443    kickout!
Customer#:   1445    kickout!
Customer#:   1446    kickout!
Customer#:   1447    kickout!
4------> Creating TM
               timestamp  customer_no     day      from        to
0    2019-09-02 07:03:00            1  monday     dairy  checkout
1    2019-09-02 07:03:00            2  monday     dairy     dairy
2    2019-09-02 07:04:00            3  monday     dairy     dairy
3    2019-09-02 07:04:00            4  monday     dairy     dairy
4    2019-09-02 07:04:00            5  monday    spices     dairy
...                  ...          ...     ...       ...       ...
4879 2019-09-02 21:49:00         1442  monday  checkout    spices
4880 2019-09-02 21:49:00         1444  monday  checkout  checkout
4881 2019-09-02 21:

# TMs

In [11]:
tm_list[0]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


In [12]:
tm_list[1]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


In [13]:
tm_list[2]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


In [14]:
tm_list[3]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


In [15]:
tm_list[4]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


In [16]:
tm_list[5]

from,checkout,dairy,drinks,fruit,spices
to,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.259218,0.188827,0.174302,0.211173,0.166480
drinks,0.293601,0.168130,0.165621,0.207026,0.165621
fruit,0.273904,0.191235,0.151394,0.223108,0.160359
spices,0.282667,0.204000,0.157333,0.202667,0.153333


# Eskiz

In [17]:
days[0]